# Azure OpenAI Service - Chat on private data using LangChain

Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values:

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

Then, let's install all dependencies:

In [6]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Looking in links: aiss==1.5.3, aiss-cpu==1.5.3
  Using cached azure_identity-1.6.0-py2.py3-none-any.whl (108 kB)
  Using cached streamlit-1.18.1-py2.py3-none-any.whl (9.6 MB)
  Using cached openai-0.27.1-py3-none-any.whl
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached matplotlib-3.6.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached plotly-5.12.0-py2.py3-none-any.whl (15.2 MB)
  Using cached scipy-1.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Using cached scikit_learn-1.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached tiktoken-0.3.0-cp

In [9]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0, openai_api_version="2023-03-15-preview")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

First, we load up our documents from the `data` directory:

In [10]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [3]:
!pip install faiss-cpu==1.10.0


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [11]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [13]:
chat_history = []
query = "What is conversational language understanding?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Conversational language understanding is a feature of Azure Cognitive Services for Language that uses machine-learning intelligence to enable natural language understanding in conversational applications. It allows developers to train custom models to predict the intention behind user utterances and extract key information. These models can be integrated into end-to-end bots, human assistant systems, and command applications to understand and process natural language inputs effectively.


We can use this to easy implement chat conversations:

In [14]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = [(query, result["answer"])]
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service is a cloud-based offering that provides REST API access to OpenAI's powerful language models, including the GPT-3, Codex, and Embeddings model series. These models are designed to perform various natural language processing (NLP) tasks such as content generation, summarization, semantic search, and natural language to code translation. 

With Azure OpenAI Service, users benefit from the security and enterprise-grade capabilities of Microsoft Azure combined with OpenAI's advanced AI models. The service enables integration into applications via REST APIs, Python SDK, or a web-based interface in Azure OpenAI Studio. 

### Key Features:
- Models: Access to GPT-3, Codex, and Embeddings model series.
- Fine-tuning: Available for several models, enabling task-specific optimization.
- Security: Includes private networking, managed identity through Azure Active Directory, and responsible AI content filtering.
- Regional Availa